# FindOpenSimulationModels

An experiment to find simulation models such as FMU and Modelica files on the open internet. I am curious how prevalent they are and whether they have inputs and outputs that would be suitable for reinforcement learning environments.

In [2]:
print("Hello Jupyter")

Hello Jupyter
